# Preprocessing Steps

This tutorial continues using the example dataset CSVs generated in the previous entry and runs them through a Deep Learning focused pipeline comprised of the following ordered steps:

* Binarizing segmentation files (if specified)
* Enforcing desired coordinate orientation (defaults to 'RAS')
* Resampling to desired voxel spacing (defaults to 1mm isotropic)
* Derivation of a skullstrip mask using [synthstrip](https://surfer.nmr.mgh.harvard.edu/docs/synthstrip/)
* Intra-study registration with [synthmorph](https://martinos.org/malte/synthmorph/) of all series within a study to the series with a desired "NormalizedSeriesDescription" (defaults to "T1Post")
* Longitudinal registration with [synthmorph](https://martinos.org/malte/synthmorph/) to first study of the same patient or to a desired atlas (if specified)
* N4 bias field correction over the skullstrip mask or a foreground foreground mask (defaults to skullstrip mask if skullstripping not disabled)
* 0 mean, unit variance normalization of volume based on statistics over the skullstrip mask
* Background normalization to 0 based on the skullstripped mask or the foreground mask (defaults to skullstrip mask if skullstripping not disabled)

In [1]:
!preprocessing brain-preprocessing --help

usage: preprocessing <command> [<args>]

The following commands are available:
    validate-installation       Check that the `preprocessing` library is installed correctly along
                                with all of its dependencies.

    dicom-dataset               Create a DICOM dataset CSV compatible with subsequent `preprocessing`
                                scripts. The final CSV provides a series level summary of the location
                                of each series alongside metadata extracted from DICOM headers.  If the
                                previous organization schems of the dataset does not enforce a DICOM
                                series being isolated to a unique directory (instances belonging to
                                multiple series must not share the same lowest level directory),
                                reorganization must be applied for NIfTI conversion.

    nifti-dataset               Create a NIfTI dataset CSV compat

In [3]:
!preprocessing brain-preprocessing \
    /autofs/space/crater_001/datasets/public/NIH_IDC_Brain/upenn_gbm_preprocessed \
    dicom_dataset_examples/upenn_gbm_dataset.csv \
    -ns \
    -c 30

Preprocessing patients:   0%|                           | 0/539 [00:00<?, ?it/s]......Clearing unnecessary files......
Finished preprocessing sub-121:
                                   SeriesInstanceUID  ...                                    PreprocessedSeg
0  1.3.6.1.4.1.14519.5.2.1.1452448862230347881864...  ...  /autofs/space/crater_001/datasets/public/NIH_I...

[1 rows x 18 columns]
Preprocessing patients:   0%|                | 1/539 [01:18<11:45:49, 78.72s/it]......Clearing unnecessary files......
Finished preprocessing sub-109:
                                   SeriesInstanceUID  ...                                    PreprocessedSeg
0  1.3.6.1.4.1.14519.5.2.1.1227166070094937926102...  ...  /autofs/space/crater_001/datasets/public/NIH_I...

[1 rows x 18 columns]
Preprocessing patients:   0%|                 | 2/539 [01:57<8:16:05, 55.43s/it]......Clearing unnecessary files......
Finished preprocessing sub-114:
                                   SeriesInstanceUID  ...        

In [4]:
from preprocessing.brain import preprocess_from_csv
help(preprocess_from_csv)

Help on function preprocess_from_csv in module preprocessing.brain.brain_preprocessing:

preprocess_from_csv(csv: pathlib.Path | str, preprocessed_dir: pathlib.Path | str, patients: Optional[Sequence[str]] = None, pipeline_key: str = 'preprocessed', registration_key: str = 'T1Post', longitudinal_registration: bool = False, atlas_target: pathlib.Path | str | None = None, registration_model: Literal['rigid', 'affine', 'affine_crop', 'joint', 'deform'] = 'affine', orientation: str = 'RAS', spacing: Sequence[float | int] = [1, 1, 1], skullstrip: bool = True, pre_skullstripped: bool = False, binarize_seg: bool = False, cpus: int = 1, verbose: bool = False, debug: bool = False) -> pandas.core.frame.DataFrame
    Preprocess all of the studies within a dataset.
    
    Parameters
    ----------
    csv: Path | str
        The path to a CSV containing an entire dataset. It must contain the following columns:  'Nifti',
        'AnonPatientID', 'AnonStudyID', 'StudyInstanceUID', 'SeriesInstanceU

In [ ]:
df = preprocess_from_csv(
    csv="dicom_dataset_examples/remind_dataset.csv",
    preprocessed_dir="/autofs/space/crater_001/datasets/public/NIH_IDC_Brain/remind_preprocessed",
    skullstrip=False,
    cpus=30
)

Preprocessing patients:   0%|                                                                                                                                                                                                                                                                                          | 0/100 [00:00<?, ?it/s]

......Clearing unnecessary files......
Finished preprocessing sub-112:
                                   SeriesInstanceUID  \
0  1.3.6.1.4.1.14519.5.2.1.2475323242339253111423...   

                                    StudyInstanceUID   PatientID  \
0  1.3.6.1.4.1.14519.5.2.1.2206833339286569121682...  ReMIND-076   

  AccessionNumber          Manufacturer StudyDate StudyDescription  \
0             NaN  Unknown manufacturer  19821225            Preop   

       SeriesDescription Modality  \
0  3D_AX_T1_postcontrast       MR   

                                              Dicoms AnonPatientID  \
0  /autofs/space/crater_001/datasets/public/NIH_I...       sub-112   

  AnonStudyID NormalizedSeriesDescription SeriesType  \
0      ses-01                      T1Post       anat   

                                               Nifti  \
0  /autofs/space/crater_001/datasets/public/NIH_I...   

                                                 Seg  \
0  /autofs/space/crater_001/datasets/pub

Preprocessing patients:   1%|██▋                                                                                                                                                                                                                                                                            | 1/100 [02:13<3:40:23, 133.57s/it]

......Clearing unnecessary files......
Finished preprocessing sub-03:
                                   SeriesInstanceUID  \
0  1.3.6.1.4.1.14519.5.2.1.1492879716425876232900...   

                                    StudyInstanceUID   PatientID  \
0  1.3.6.1.4.1.14519.5.2.1.8778058275794785106466...  ReMIND-013   

  AccessionNumber          Manufacturer StudyDate StudyDescription  \
0             NaN  Unknown manufacturer  19821225            Preop   

       SeriesDescription Modality  \
0  3D_AX_T1_postcontrast       MR   

                                              Dicoms AnonPatientID  \
0  /autofs/space/crater_001/datasets/public/NIH_I...        sub-03   

  AnonStudyID NormalizedSeriesDescription SeriesType  \
0      ses-02                      T1Post       anat   

                                               Nifti  \
0  /autofs/space/crater_001/datasets/public/NIH_I...   

                                                 Seg  \
0  /autofs/space/crater_001/datasets/publ

Preprocessing patients:   2%|█████▍                                                                                                                                                                                                                                                                         | 2/100 [03:31<2:44:28, 100.70s/it]

...

Preprocessing patients:  70%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                  | 70/100 [31:16<10:56, 21.87s/it]

......Clearing unnecessary files......
Finished preprocessing sub-56:
                                   SeriesInstanceUID  \
0  1.3.6.1.4.1.14519.5.2.1.4367000335281376953887...   

                                    StudyInstanceUID   PatientID  \
0  1.3.6.1.4.1.14519.5.2.1.1847320936718690184530...  ReMIND-042   

  AccessionNumber          Manufacturer StudyDate StudyDescription  \
0             NaN  Unknown manufacturer  19821225            Preop   

       SeriesDescription Modality  \
0  3D_AX_T1_postcontrast       MR   

                                              Dicoms AnonPatientID  \
0  /autofs/space/crater_001/datasets/public/NIH_I...        sub-56   

  AnonStudyID NormalizedSeriesDescription SeriesType  \
0      ses-01                      T1Post       anat   

                                               Nifti  \
0  /autofs/space/crater_001/datasets/public/NIH_I...   

                                                 Seg  \
0  /autofs/space/crater_001/datasets/publ

Preprocessing patients:  71%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                               | 71/100 [32:31<18:23, 38.06s/it]

......Clearing unnecessary files......
Finished preprocessing sub-91:
                                   SeriesInstanceUID  \
0  1.3.6.1.4.1.14519.5.2.1.2119189330391126633248...   

                                    StudyInstanceUID   PatientID  \
0  1.3.6.1.4.1.14519.5.2.1.1629322430880633286088...  ReMIND-011   

  AccessionNumber          Manufacturer StudyDate StudyDescription  \
0             NaN  Unknown manufacturer  19821225            Preop   

       SeriesDescription Modality  \
0  3D_AX_T1_postcontrast       MR   

                                              Dicoms AnonPatientID  \
0  /autofs/space/crater_001/datasets/public/NIH_I...        sub-91   

  AnonStudyID NormalizedSeriesDescription SeriesType  \
0      ses-01                      T1Post       anat   

                                               Nifti  \
0  /autofs/space/crater_001/datasets/public/NIH_I...   

                                                 Seg  \
0  /autofs/space/crater_001/datasets/publ

Preprocessing patients:  72%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                            | 72/100 [33:44<22:38, 48.50s/it]